In [1]:

import torch
from transformer_lens import HookedTransformer
import os
import pandas as pd
from IPython.display import display, HTML

/workspace/MATS-research/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

# Check if a GPU is available and set the device
if torch.cuda.is_available():
    device = "cuda"
    print(f"GPU detected: {torch.cuda.get_device_name(0)}")
    torch.cuda.empty_cache()
else:
    device = "cpu"
    print("No GPU detected, using CPU. This will be very slow.")


GPU detected: NVIDIA GeForce RTX 3090


In [3]:

# The model you specified
model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
print(f"\n--- Loading Model: {model_name} ---")
print("This will download and load ~16 GB of model weights. This may take several minutes.")

# Load the model using HookedTransformer.
# We use bfloat16 for performance, which is well-supported by 3090 GPUs.
model = HookedTransformer.from_pretrained(
    model_name,
    device=device,
    torch_dtype=torch.bfloat16,
)



--- Loading Model: meta-llama/Meta-Llama-3-8B-Instruct ---
This will download and load ~16 GB of model weights. This may take several minutes.


`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 4/4 [00:00<00:00, 101.94it/s]


Loaded pretrained model meta-llama/Meta-Llama-3-8B-Instruct into HookedTransformer


In [4]:

# --- 2. Helper Functions for Data Processing ---

def split_conversation(text: str, user_identifier="HUMAN:", ai_identifier="ASSISTANT:") -> list[dict]:
    """
    Parses the conversation text into a list of dictionaries,
    each with a 'role' and 'content' key.
    """
    messages = []
    lines = text.strip().split('\n')
    
    current_role = None
    current_content = ""

    for line in lines:
        if line.startswith(user_identifier):
            if current_role: # Save the previous message
                messages.append({'role': current_role, 'content': current_content.strip()})

            current_role = 'user'
            current_content = line[len(user_identifier):].strip()
        elif line.startswith(ai_identifier):
            if current_role:
                messages.append({'role': current_role, 'content': current_content.strip()})

            current_role = 'assistant'
            current_content = line[len(ai_identifier):].strip()
        else:
            # Append to the content of the last speaker
            current_content += "\n" + line
    
    # Add the last message
    if current_role and current_content:
        messages.append({'role': current_role, 'content': current_content.strip()})
        
    return messages